<a href="https://colab.research.google.com/github/JoaoMiguel-A01/Projeto_VerificaPDF/blob/main/Ol%C3%A1%2C_este_%C3%A9_o_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
import csv
import json
import re
from datetime import datetime
import calendar
from collections import defaultdict
import xml.etree.ElementTree as ET

# Função para remover tudo exceto números
def somente_numeros(cnpj):
    return re.sub(r'\D', '', cnpj)

# Função para normalizar os dados
def normalizar_dado(dado):
    if isinstance(dado, str):
        return dado.strip().replace(".", "").replace(",", ".").upper()
    return dado

# Função para limpar e normalizar os nomes
def limpar_nome(nome):
    return re.sub(r"\s+", " ", nome.replace("\n", " ").strip())

def comparar_datas(data_xml, data_planilha):
    try:
        # Converter ambos os formatos para objetos datetime
        data_formatada_xml = datetime.strptime(data_xml, "%Y-%m-%d")  # ISO (aaaa-mm-dd)
        data_formatada_planilha = datetime.strptime(data_planilha, "%d/%m/%Y")  # dd/mm/aaaa
        # Comparar as datas convertidas
        return data_formatada_xml == data_formatada_planilha
    except (ValueError, TypeError):
        print(f"[ERRO] Não foi possível comparar as datas: '{data_xml}' e '{data_planilha}'")
        return False  # Retorna falso se houver erro na conversão


# Função para converter a planilha em um arquivo JSON
def converter_planilha_para_json(caminho_planilha, caminho_json):
    dados = []
    with open(caminho_planilha, mode="r", encoding="latin-1") as arquivo_csv:
        leitor = csv.reader(arquivo_csv, delimiter=";")
        for i, linha in enumerate(leitor, start=1):
            if len(linha) < 18:  # Verificar se há colunas suficientes na linha
                print(f"[ERRO] Linha {i} ignorada: número insuficiente de colunas ({len(linha)}).")
                continue
            try:
                dados.append({
                    "N_PFE": normalizar_dado(linha[0]),
                    "nome_emissor": limpar_nome(linha[2]),
                    "uf_emissor": normalizar_dado(linha[3]),
                    "cnpj_emissor": normalizar_dado(linha[4]),
                    "nome_destinatario": limpar_nome(linha[5]),
                    "uf_destinatario": normalizar_dado(linha[6]),
                    "cnpj_destinatario": normalizar_dado(linha[7]),
                    "data_emissao": normalizar_dado(linha[9]),
                    "data_vencimento": normalizar_dado(linha[11]),
                    "quant": normalizar_dado(linha[12]),
                    "valor_total": normalizar_dado(linha[19]),
                    "cfop": normalizar_dado(linha[20])
                })
            except IndexError as e:
                print(f"[ERRO] Linha {i} com erro: {e}. Conteúdo: {linha}")

    # Salvar os dados extraídos da planilha no formato JSON
    with open(caminho_json, "w", encoding="utf-8") as json_file:
        json.dump(dados, json_file, ensure_ascii=False, indent=4)
    print(f"[INFO] Planilha convertida com sucesso. Dados salvos em {caminho_json}.")

# Função para extrair dados do XMLimport xml.etree.ElementTree as ET

def extrair_dados_xml(caminho_xml):
    dados = {
        "arquivo_original": caminho_xml.split("/")[-1],
        "nome": "",
        "data_emissao": "",
        "data_vencimento": "",
        "valor_total": "",
        "uf": "",
        "numero_nf": "",
        "cfop": "",
        "quant": "",
        "cnpj_destinatario": "",
        "nome_emissor": "",
        "cnpj_emissor": ""
    }

    print(f"[INFO] Processando arquivo XML: {caminho_xml}")

    try:
        # Ler e interpretar o arquivo XML
        tree = ET.parse(caminho_xml)
        root = tree.getroot()

        # Obter namespace do XML
        namespace = {"nfe": "http://www.portalfiscal.inf.br/nfe"}

        # Capturar Número NF
        numero_nf = root.find(".//nfe:ide/nfe:nNF", namespace)
        if numero_nf is not None:
            dados["numero_nf"] = numero_nf.text.strip()
        else:
            print("[DEBUG] Tag <nNF> não encontrada.")

        # Capturar Nome do Emitente
        nome_emissor = root.find(".//nfe:emit/nfe:xNome", namespace)
        if nome_emissor is not None:
            dados["nome_emissor"] = nome_emissor.text.strip()
        else:
            print("[DEBUG] Tag <xNome> não encontrada no emitente.")

        # Capturar CNPJ do Emitente
        cnpj_emissor = root.find(".//nfe:emit/nfe:CNPJ", namespace)
        if cnpj_emissor is not None:
            dados["cnpj_emissor"] = cnpj_emissor.text.strip()
        else:
            print("[DEBUG] Tag <CNPJ> não encontrada no emitente.")

        # Capturar Nome do Destinatário
        nome_destinatario = root.find(".//nfe:dest/nfe:xNome", namespace)
        if nome_destinatario is not None:
            dados["nome"] = nome_destinatario.text.strip()
        else:
            print("[DEBUG] Tag <xNome> não encontrada no destinatário.")

        # Capturar CNPJ do Destinatário
        cnpj_destinatario = root.find(".//nfe:dest/nfe:CNPJ", namespace)
        if cnpj_destinatario is not None:
            dados["cnpj_destinatario"] = cnpj_destinatario.text.strip()
        else:
            print("[DEBUG] Tag <CNPJ> não encontrada no destinatário.")

        #Capturar Data de Emissão
        data_emissao = root.find(".//nfe:ide/nfe:dhEmi", namespace)
        if data_emissao is not None:
            dados["data_emissao"] = data_emissao.text[:10]  # Pega apenas a parte da data (YYYY-MM-DD)
        else:
            print("[DEBUG] Tag <dhEmi> não encontrada.")

        # Capturar Valor Total
        valor_total = root.find(".//nfe:total/nfe:ICMSTot/nfe:vNF", namespace)
        if valor_total is not None:
            dados["valor_total"] = valor_total.text.strip()
        else:
            print("[DEBUG] Tag <vNF> não encontrada.")

        # Capturar UF do Destinatário
        uf = root.find(".//nfe:dest/nfe:enderDest/nfe:UF", namespace)
        if uf is not None:
            dados["uf"] = uf.text.strip()
        else:
            print("[DEBUG] Tag <UF> não encontrada no destinatário.")

        # Capturar CFOP
        cfop = root.find(".//nfe:det/nfe:prod/nfe:CFOP", namespace)
        if cfop is not None:
            dados["cfop"] = cfop.text.strip()
        else:
            print("[DEBUG] Tag <CFOP> não encontrada.")

        # Capturar Quantidade de Produtos
        quant = root.find(".//nfe:det/nfe:prod/nfe:qCom", namespace)
        if quant is not None:
            dados["quant"] = quant.text.strip()
        else:
            print("[DEBUG] Tag <qCom> não encontrada.")

        # Capturar Data de Vencimento (se disponível)
        data_vencimento = root.find(".//nfe:cobr/nfe:dup/nfe:dVenc", namespace)
        if data_vencimento is not None:
            dados["data_vencimento"] = data_vencimento.text.strip()
        else:
            print("[DEBUG] Tag <dVenc> não encontrada.")

        print(f"[SUCESSO] Dados extraídos: {dados}")
    except ET.ParseError as e:
        print(f"[ERRO] Problema ao processar o XML: {e}")
    except Exception as e:
        print(f"[ERRO] Falha ao processar {caminho_xml}: {e}")

    return dados


# Função para converter os arquivos XML em um arquivo JSON
def converter_xml_para_json(pasta_xml, caminho_json):
    dados_xml = []
    for nome_arquivo in os.listdir(pasta_xml):
        if nome_arquivo.endswith(".xml"):
            caminho_xml = os.path.join(pasta_xml, nome_arquivo)
            dados_xml.append({
                "arquivo_original": nome_arquivo,  # Guarda o nome original do arquivo
                **extrair_dados_xml(caminho_xml)  # Extrai dados e adiciona ao dicionário
            })

    # Salvar os dados extraídos dos XMLs no formato JSON
    with open(caminho_json, "w", encoding="utf-8") as json_file:
        json.dump(dados_xml, json_file, ensure_ascii=False, indent=4)
    print(f"[INFO] XMLs convertidos com sucesso. Dados salvos em {caminho_json}.")

# Função para carregar dados de um arquivo JSON
def carregar_dados_json(caminho_json): # defining the missing function
    with open(caminho_json, 'r', encoding='utf-8') as f:
        return json.load(f)

# Função para validar e renomear arquivos com base nos dados extraídos do XML
def validar_e_renomear_arquivos(caminho_json_planilha, caminho_json_xml, pasta_arquivos, pasta_saida, pasta_nao_validados, caminho_txt_relatorio):
    # Carregar dados da planilha e do XML em JSON
    dados_planilha = carregar_dados_json(caminho_json_planilha)
    dados_xml = carregar_dados_json(caminho_json_xml)
    nao_validados = []  # Lista para arquivos não validados
    linhas_relatorio = []  # Linhas para o relatório TXT

    # Agrupar registros da planilha por CNPJ
    registros_por_cnpj = defaultdict(list)
    for registro in dados_planilha:
        cnpj_destinatario = somente_numeros(registro.get("cnpj_destinatario", ""))
        registros_por_cnpj[cnpj_destinatario].append(registro)

    while dados_xml:
        xml_atual = dados_xml.pop(0)  # Pega o primeiro registro XML
        arquivo_original = xml_atual.get("arquivo_original", None)
        if not arquivo_original:
            mensagem = f"[AVISO] Arquivo original ausente no registro do XML: {xml_atual}"
            print(mensagem)
            linhas_relatorio.append(mensagem)
            nao_validados.append(xml_atual)
            continue

        cnpj_xml = somente_numeros(xml_atual.get("cnpj_destinatario", ""))
        registros_cnpj = registros_por_cnpj.get(cnpj_xml, [])

        if not registros_cnpj:
            mensagem = f"[FALHA] Nenhum registro correspondente ao CNPJ do XML: {arquivo_original}"
            print(mensagem)
            linhas_relatorio.append(mensagem)
            nao_validados.append(xml_atual)
            continue

        validado = False
        melhor_registro = None
        menor_diferenca = float("inf")  # Para encontrar o registro mais próximo

        # Comparar o XML com os registros do mesmo CNPJ
        for registro_planilha in registros_cnpj:
            # Comparar os campos
            diferencas = [
                abs((float(xml_atual.get("valor_total", 0)) - float(registro_planilha.get("valor_total", 0)))) < 0.01,
                abs((float(xml_atual.get("quant", 0)) - float(registro_planilha.get("quant", 0)))) < 0.01,
                comparar_datas(xml_atual.get("data_vencimento"), registro_planilha.get("data_vencimento")),
                xml_atual.get("cfop") == registro_planilha.get("cfop"),
                xml_atual.get("uf") == registro_planilha.get("uf_destinatario"),
            ]

            # Priorizar o registro com menos divergências
            total_diferencas = sum(1 for diff in diferencas if not diff)
            if total_diferencas < menor_diferenca:
                menor_diferenca = total_diferencas
                melhor_registro = registro_planilha

        # Validar com o melhor registro
        if melhor_registro and menor_diferenca == 0:  # Exige correspondência perfeita
            validado = True
            data_vencimento = datetime.strptime(xml_atual["data_vencimento"], "%Y-%m-%d")  # VALIDAR ISSO AQUI DPS
            mes_alfanumerico = f"{data_vencimento.month:02}{str(data_vencimento.year)[-2:]}"
            mes_referencia = calendar.month_abbr[data_vencimento.month - 1].upper() + str(data_vencimento.year)[-2:]
            novo_nome = f"{mes_alfanumerico}_{registro_planilha['nome_destinatario']}_{mes_referencia}_NF-{xml_atual.get('numero_nf', 'SEM-NF')}.xml"

            # Renomear e mover o arquivo
            caminho_origem = os.path.join(pasta_arquivos, arquivo_original)
            caminho_destino = os.path.join(pasta_saida, novo_nome)
            print(f"[DEBUG] Tentando mover arquivo de {caminho_origem} para {caminho_destino}")
            if os.path.exists(caminho_origem):
                os.rename(caminho_origem, caminho_destino)
                print(f"[VALIDADO] Arquivo renomeado para: {novo_nome}")
            else:
                mensagem = f"[ERRO] Arquivo XML não encontrado: {arquivo_original}"
                print(mensagem)
                linhas_relatorio.append(mensagem)

        if not validado:
            mensagem = f"[FALHA] XML não validado: {arquivo_original}"
            print(mensagem)
            linhas_relatorio.append(mensagem)

            # Log detalhado das divergências
            if melhor_registro:
                detalhes = f"[CONFLITO] Comparado com registro errado: {melhor_registro}"
                linhas_relatorio.append(detalhes)

            nao_validados.append(xml_atual)

    # Gerar arquivo TXT do relatório com mais detalhes relevantes
    with open(caminho_txt_relatorio, "w", encoding="utf-8") as arquivo_relatorio:
        # Escrever mensagens gerais do log
        for linha in linhas_relatorio:
            arquivo_relatorio.write(linha + "\n")

        # Adicionar detalhes específicos das falhas
        arquivo_relatorio.write("\n=== Relatório de XMLs Não Validados ===\n")
        for xml in nao_validados:
            arquivo_relatorio.write(f"\n[FALHA] XML não validado: {xml.get('arquivo_original', 'Arquivo desconhecido')}\n")
            print(f"[FALHA] Comparação falhou para o XML: {xml.get('arquivo_original', 'Arquivo desconhecido')}")  # Exibir no console

            melhor_registro = xml.get("melhor_registro")
            if melhor_registro:
                arquivo_relatorio.write("    Melhor registro encontrado para comparação:\n")
                arquivo_relatorio.write(f"      - Nome Destinatário: {melhor_registro.get('nome_destinatario', 'Não encontrado')}\n")
                arquivo_relatorio.write(f"      - CNPJ Destinatário: {melhor_registro.get('cnpj_destinatario', 'Não encontrado')}\n")
                arquivo_relatorio.write(f"      - Valor Total: {melhor_registro.get('valor_total', 'Não encontrado')}\n")

                print(f"    Comparado com o registro da planilha: {melhor_registro}")  # Exibir no console

                # Detalhar campos divergentes
                divergencias = []
                for campo, valor_xml in xml.get("diferencas", {}).items():
                    valor_planilha = melhor_registro.get(campo, "Não encontrado")
                    if valor_xml != valor_planilha:
                        divergencias.append(f"    - {campo}: XML='{valor_xml}', Planilha='{valor_planilha}'")
                        print(f"    Divergência no campo '{campo}': XML='{valor_xml}', Planilha='{valor_planilha}'")  # Exibir no console

                if divergencias:
                    arquivo_relatorio.write("    Detalhes das divergências:\n")
                    for divergencia in divergencias:
                        arquivo_relatorio.write(divergencia + "\n")
                else:
                    arquivo_relatorio.write("    Nenhuma divergência significativa encontrada.\n")
            else:
                arquivo_relatorio.write("    Nenhum registro correspondente encontrado na planilha.\n")
                print("    Nenhum registro correspondente encontrado na planilha.")  # Exibir no console



    # Mover XMLs não validados para a pasta separada
    for xml in nao_validados:
        arquivo_origem = os.path.join(pasta_arquivos, xml.get("arquivo_original", ""))
        if os.path.exists(arquivo_origem):
            destino_nao_validado = os.path.join(pasta_nao_validados, xml.get("arquivo_original", ""))
            try:
                os.rename(arquivo_origem, destino_nao_validado)
                print(f"[NAO VALIDADO] Arquivo movido para a pasta 'naoValidados': {destino_nao_validado}")
            except Exception as e:
                print(f"[ERRO] Não foi possível mover o arquivo '{arquivo_origem}' para '{destino_nao_validado}': {e}")
        else:
            mensagem = f"[ERRO] Arquivo não encontrado para mover: {arquivo_origem}"
            print(mensagem)
            linhas_relatorio.append(mensagem)

# Configuração principal
pasta_xml = "./naoValidados"  # Nova pasta para arquivos XML não validados
pasta_saida = "./validados"
pasta_nao_corresponde = "./naoCorresponde"
caminho_planilha = "./PlanilhaControleL - Copia.csv"
caminho_json_planilha = "./dados_planilha.json"
caminho_json_xml = "./dados_xml.json"  # Nome ajustado para refletir XML
caminho_txt_relatorio = "./relatorio.txt"

# Garantir que as pastas existam
os.makedirs(pasta_xml, exist_ok=True)  # Atualizado para refletir a mudança para XML
os.makedirs(pasta_saida, exist_ok=True)
os.makedirs(pasta_nao_corresponde, exist_ok=True)

# Chamadas das funções principais
converter_planilha_para_json(caminho_planilha, caminho_json_planilha)
converter_xml_para_json(pasta_xml, caminho_json_xml)  # Atualizado para converter XML em JSON
validar_e_renomear_arquivos(caminho_json_planilha, caminho_json_xml, pasta_xml, pasta_saida, pasta_nao_corresponde, caminho_txt_relatorio)  # Nova função para XML

[INFO] Planilha convertida com sucesso. Dados salvos em ./dados_planilha.json.
[INFO] Processando arquivo XML: ./naoValidados/33250330248458000125550010001720521017669007.xml
[DEBUG] Tag <dVenc> não encontrada.
[SUCESSO] Dados extraídos: {'arquivo_original': '33250330248458000125550010001720521017669007.xml', 'nome': 'COMEL - COMERCIALIZADORA DE ENERGIA LIASA LTDA', 'data_emissao': '2025-03-10', 'data_vencimento': '', 'valor_total': '1218324.24', 'uf': 'SP', 'numero_nf': '172052', 'cfop': '6251', 'quant': '10584.0000', 'cnpj_destinatario': '39953546000100', 'nome_emissor': 'ENEL TRADING BRASIL S/A', 'cnpj_emissor': '30248458000125'}
[INFO] Processando arquivo XML: ./naoValidados/50250323096269000380550010000152871312565350.xml
[SUCESSO] Dados extraídos: {'arquivo_original': '50250323096269000380550010000152871312565350.xml', 'nome': 'COMEL - COMERCIALIZADORA DE ENERGIA LIASA LTDA', 'data_emissao': '2025-03-06', 'data_vencimento': '2025-03-12', 'valor_total': '307392.42', 'uf': 'SP', 'n